![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)
(https://colab.research.google.com/github/ricardokleinklein/NLP_GenMods/blob/main/TTS.ipynb)

# Modelos Generativos

## WaveNet - Audio

Creado por *Ricardo Kleinlein* para [Saturdays.AI](https://saturdays.ai/).

Disponible bajo una licencia [Creative Commons](https://creativecommons.org/licenses/by/4.0/).

---

## Sobre el uso de Jupyter Notebooks

Este notebook ha sido implementado en Python, pero para su ejecución no es
necesario conocer el lenguaje en profundidad. Solamente se debe ejecutar cada
una de las celdas, teniendo en cuenta que hay que ejecutar una celda a la vez
y secuencialmente, tal y como figuran en orden de aparición.

Para ejecutar cada celda pulse en el botón ▶ en la esquina superior izquierda
de cada celda. Mientras se esté ejecutando ese fragmento de código,
el botón estará girando. En caso de querer detener dicha ejecución, pulse
nuevamente sobre este botón mientras gira y la ejecución se detendrá. En caso
de que la celda tenga alguna salida (texto, gráficos, etc) será mostrada
justo después de esta y antes de mostrar la siguiente celda. El notebook
estará guiado con todas las explicaciones necesarias, además irá acompañado
por comentarios en el código para facilitar su lectura.

En caso de tener alguna duda, anótela. Dedicaremos un tiempo a plantear y
resolver la mayoría delas dudas que puedan aparecer.


## Objetivo del notebook

Tener una intuición acerca de cómo podemos generar habla sintética a partir
de mensajes en texto mediante la red neuronal profunda WaveNet.

### Disclaimer

Trabajar con el modelo completo requeriría GPUs, toneladas de cálculos y
mucho tiempo. El WaveNet original tarda hasta 5 minutos en generar cada
segundo de audio, por lo que aquí vamos a trabajar mediante la API de Google
 Translate.

## Sobre el modelo

Vamos a emplear la versión de WaveNet que utiliza Google Translate por
defecto para dar voz a las traducciones generadas. En el modelo original,
mostrado abajo, existen muchas maneras de modificar la voz producida, la
cual ni siquiera debe circunscribirse a un texto en concreto (véase [el blog
original](https://deepmind.com/blog/article/wavenet-generative-model-raw
-audio) y las muestras de audio que adjuntan).

![Estructura generativa Wavenet](./assets/wavenet.gif)

Procedimientos habituales son "condicionar" la señal de entrada para generar
 voces con acentos concretos, cambiar el sexo de la voz, o la que
 realizaremos nosotros, escribir explícitamente qué queremos que diga.

## Importar las librerías necesarias

In [7]:
!pip install gTTS
from gtts import gTTS
from IPython.display import Audio

In [9]:
# Vamos a ver ejemplos de generación en inglés y en español, pero existe un
# gama enorme de idiomas en los que trabajar. Véase:
# https://gtts.readthedocs.io/en/latest/module.html#languages-gtts-lang

spa_example = gTTS('Saludos de parte de todo el equipo de Saturdays.AI',
                   lang="es")
spa_example.save('/content/SPA-example.mp3')
Audio('/content/SPA-example.mp3', autoplay=True)

eng_example = gTTS('Greetings on behalf of the team of Saturdays.AI',
                   lang='en')
eng_example.save('/content/ENG-example.mp3')
Audio('/content/ENG-example.mp3', autoplay=True)

## DeepVoice3

DeepVoice es un modelo de Text-To-Speech Synthesis centrado únicamente en la
 combinación de capas convolucionales y atención.

In [ ]:
import os
from os.path import exists, join, expanduser

# Clone
name = "deepvoice3_pytorch"
if not exists(name):
  ! git clone https://github.com/r9y9/$name
# Change working directory to the project dir
os.chdir(join(expanduser("~"), name))
!git checkout 7a10ac6763eda92595e257543494b6a95f64229b --quiet
# Install dependencices
!pip install -q -e '.[bin]'

In [ ]:
%pylab inline
! pip install -q librosa nltk

import torch
import numpy as np
import librosa
import librosa.display
import IPython
from IPython.display import Audio
# need this for English text processing frontend
import nltk
! python -m nltk.downloader cmudict

Descargamos el modelo pre-entrenado

In [ ]:
preset = "20180505_deepvoice3_ljspeech.json"
checkpoint_path = "20180505_deepvoice3_checkpoint_step000640000.pth"

In [ ]:
if not exists(preset):
  !curl -O -L "https://www.dropbox.com/s/0ck82unm0bo0rxd/20180505_deepvoice3_ljspeech.json"
if not exists(checkpoint_path):
  !curl -O -L "https://www.dropbox.com/s/5ucl9remrwy5oeg/20180505_deepvoice3_checkpoint_step000640000.pth"

## Sintetizamos con este nuevo modelo

Primero definimos los hiperparámetros del modelo

In [ ]:
import hparams
import json

# Load parameters from preset
with open(preset) as f:
  hparams.hparams.parse_json(f.read())

# Inject frontend text processor
import synthesis
import train
from deepvoice3_pytorch import frontend
synthesis._frontend = getattr(frontend, "en")
train._frontend =  getattr(frontend, "en")

# alises
fs = hparams.hparams.sample_rate
hop_length = hparams.hparams.hop_size

### Definimos algunas funciones prácticas

In [ ]:
def tts(model, text, p=0, speaker_id=None, fast=True, figures=True):
  from synthesis import tts as _tts
  waveform, alignment, spectrogram, mel = _tts(model, text, p, speaker_id, fast)
  if figures:
      visualize(alignment, spectrogram)
  IPython.display.display(Audio(waveform, rate=fs))

def visualize(alignment, spectrogram):
  label_fontsize = 16
  figure(figsize=(16,16))

  subplot(2,1,1)
  imshow(alignment.T, aspect="auto", origin="lower", interpolation=None)
  xlabel("Decoder timestamp", fontsize=label_fontsize)
  ylabel("Encoder timestamp", fontsize=label_fontsize)
  colorbar()

  subplot(2,1,2)
  librosa.display.specshow(spectrogram.T, sr=fs,
                           hop_length=hop_length, x_axis="time", y_axis="linear")
  xlabel("Time", fontsize=label_fontsize)
  ylabel("Hz", fontsize=label_fontsize)
  tight_layout()
  colorbar()

### Cargamos el modelo

In [ ]:
from train import build_model
from train import restore_parts, load_checkpoint

model = build_model()
model = load_checkpoint(checkpoint_path, model, None, True)

### Vamos a generar audio a partir de frases

In [ ]:
texts = [
    "Scientists at the CERN laboratory say they have discovered a new particle.",
    "There's a way to measure the acute emotional intelligence that has never gone out of style.",
    "President Trump met with other leaders at the Group of 20 conference.",
    "The Senate's bill to repeal and replace the Affordable Care Act is now imperiled.",
    "Generative adversarial network or variational auto-encoder.",
    "The buses aren't the problem, they actually provide a solution.",
    "peter piper picked a peck of pickled peppers how many peppers did peter piper pick.",
    "Some have accepted this as a miracle without any physical explanation.",
]

for idx, text in enumerate(texts):
  print(idx, text)
  tts(model, text, figures=False)